In [2]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

 35%|█████████████▎                        | 9.00M/25.7M [00:00<00:00, 85.9MB/s]
100%|███████████████████████████████████████| 25.7M/25.7M [00:00<00:00, 164MB/s]


In [6]:
path_to_zip_file = '/home/jupyter/imdb-dataset-of-50k-movie-reviews.zip'
directory_to_extract_to = '/home/jupyter/dataset'

import zipfile
with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
    zip_ref.extractall(directory_to_extract_to)

In [74]:
import pandas as pd

imdb_data = pd.read_csv('./dataset/IMDB Dataset.csv')

In [75]:
imdb_data

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


# 데이터 전처리

In [76]:
print(imdb_data.sentiment)

0        positive
1        positive
2        positive
3        negative
4        positive
           ...   
49995    positive
49996    negative
49997    negative
49998    negative
49999    negative
Name: sentiment, Length: 50000, dtype: object


In [77]:
imdb_data.sentiment = imdb_data.sentiment.apply(lambda x: 0 if x == 'positive' else 1)
print(imdb_data.sentiment)        

0        0
1        0
2        0
3        1
4        0
        ..
49995    0
49996    1
49997    1
49998    1
49999    1
Name: sentiment, Length: 50000, dtype: int64


In [78]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

maxlen = 100
max_words = 10000

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(imdb_data.review)
sequences = tokenizer.texts_to_sequences(imdb_data.review)
print('Found %s unique tokens.' % len(tokenizer.word_index))

data = pad_sequences(sequences, maxlen=maxlen)
# #Tokenization of text
# tokenizer=ToktokTokenizer()
# #Setting English stopwords
# stopword_list=nltk.corpus.stopwords.words('english')

Found 124252 unique tokens.


In [79]:
X_train = data[:40000]
y_train = imdb_data.sentiment[:40000]

X_test = data[40000:]
y_test = imdb_data.sentiment[40000:]

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(40000, 100) (40000,)
(10000, 100) (10000,)


In [80]:
print('first train review :', X_train[0])
print('first train sentiment :', y_train[0])

first train review : [ 123  210 3241   68   14   34 1637    9   13 2239   10  413  131   10
   13 1592   15    9   18   14   10  287   51   10 1417    3 1280   15
 3184    2  189    5    1  299 2046    4 2150  570   21   39  570   18
 7658 7154 5010   26 2983   41   15    3 6904  504   20  642    2   76
  243   16    9   69 7598  651  710 6904  109  662   82 1208  693    5
   65  574    4  920 2021   38 1208  559  147 3184   22  200  426 3819
   16   48    6 3314  805 1603   43   22   67   76    8 1228   16  125
 4103  486]
first train sentiment : 0


In [81]:
import numpy as np

print("Frequency of each label")
unique_elements, counts_elements = np.unique(y_train, return_counts=True)
for unique, counts in zip(unique_elements, counts_elements):
    print("%s: %d" % (unique, counts))

Frequency of each label
0: 19993
1: 20007


In [82]:
print(y_train[:5])

0    0
1    0
2    0
3    1
4    0
Name: sentiment, dtype: int64


In [84]:
y_train = np.asarray(y_train, dtype='float32')
y_test = np.asarray(y_test, dtype='float32')

print(y_train[:5])

[0. 0. 0. 1. 0.]


## CNN 분류 모델

In [55]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dropout, Conv1D, GlobalMaxPooling1D, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model

embedding_dim = 256 # 임베딩 벡터의 차원
dropout_ratio = 0.3 # 드롭아웃 비율
num_filters = 256 # 커널의 수
kernel_size = 3 # 커널의 크기
hidden_units = 128 # 뉴런의 수

model = Sequential()
model.add(Embedding(10000, embedding_dim))
model.add(Dropout(dropout_ratio))
model.add(Conv1D(num_filters, kernel_size, padding='valid', activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(hidden_units, activation='relu'))
model.add(Dropout(dropout_ratio))
model.add(Dense(1, activation='sigmoid'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test), callbacks=[es, mc])

Epoch 1/20


2022-11-21 06:41:14.813232: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8200


1247/1250 [============================>.] - ETA: 0s - loss: 0.3868 - acc: 0.8166
Epoch 1: val_acc improved from -inf to 0.87550, saving model to best_model.h5
1250/1250 [==============================] - 18s 5ms/step - loss: 0.3867 - acc: 0.8167 - val_loss: 0.2925 - val_acc: 0.8755
Epoch 2/20
1244/1250 [============================>.] - ETA: 0s - loss: 0.2243 - acc: 0.9107
Epoch 2: val_acc improved from 0.87550 to 0.87760, saving model to best_model.h5
1250/1250 [==============================] - 7s 5ms/step - loss: 0.2242 - acc: 0.9107 - val_loss: 0.2918 - val_acc: 0.8776
Epoch 3/20
1250/1250 [==============================] - ETA: 0s - loss: 0.1182 - acc: 0.9574
Epoch 3: val_acc did not improve from 0.87760
1250/1250 [==============================] - 6s 5ms/step - loss: 0.1182 - acc: 0.9574 - val_loss: 0.3516 - val_acc: 0.8756
Epoch 4/20
1242/1250 [============================>.] - ETA: 0s - loss: 0.0628 - acc: 0.9775
Epoch 4: val_acc did not improve from 0.87760
1250/1250 [=======

In [56]:
loaded_model = load_model('best_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

313/313 [==============================] - 1s 3ms/step - loss: 0.2918 - acc: 0.8776

 테스트 정확도: 0.8776


## RNN 분류 모델

In [57]:
import re
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Embedding
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model

embedding_dim = 100
hidden_units = 128

model = Sequential()
model.add(Embedding(10000, embedding_dim))
model.add(GRU(hidden_units))
model.add(Dense(1, activation='sigmoid'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('GRU_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=15, callbacks=[es, mc], batch_size=64, validation_split=0.2)

Epoch 1/15
496/500 [============================>.] - ETA: 0s - loss: 0.4552 - acc: 0.7990
Epoch 1: val_acc improved from -inf to 0.85688, saving model to GRU_model.h5
500/500 [==============================] - 6s 8ms/step - loss: 0.4541 - acc: 0.7997 - val_loss: 0.3374 - val_acc: 0.8569
Epoch 2/15
495/500 [============================>.] - ETA: 0s - loss: 0.2851 - acc: 0.8840
Epoch 2: val_acc improved from 0.85688 to 0.86862, saving model to GRU_model.h5
500/500 [==============================] - 4s 8ms/step - loss: 0.2849 - acc: 0.8840 - val_loss: 0.3067 - val_acc: 0.8686
Epoch 3/15
499/500 [============================>.] - ETA: 0s - loss: 0.2297 - acc: 0.9104
Epoch 3: val_acc improved from 0.86862 to 0.87650, saving model to GRU_model.h5
500/500 [==============================] - 4s 8ms/step - loss: 0.2297 - acc: 0.9104 - val_loss: 0.2962 - val_acc: 0.8765
Epoch 4/15
495/500 [============================>.] - ETA: 0s - loss: 0.1929 - acc: 0.9264
Epoch 4: val_acc did not improve fro

In [58]:
loaded_model = load_model('GRU_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

313/313 [==============================] - 2s 4ms/step - loss: 0.2990 - acc: 0.8729

 테스트 정확도: 0.8729


## CNN 분류 모델 (optimizer 변경)

In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dropout, Conv1D, GlobalMaxPooling1D, Dense, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model

embedding_dim = 256 # 임베딩 벡터의 차원
dropout_ratio = 0.3 # 드롭아웃 비율
num_filters = 256 # 커널의 수
kernel_size = 3 # 커널의 크기
hidden_units = 128 # 뉴런의 수

model = Sequential()
model.add(Embedding(10000, embedding_dim))
model.add(Dropout(dropout_ratio))
model.add(Conv1D(num_filters, kernel_size, padding='valid', activation='relu'))
model.add(BatchNormalization())
model.add(Conv1D(num_filters, kernel_size, padding='valid', activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(hidden_units, activation='relu'))
model.add(Dropout(dropout_ratio))
model.add(Dense(1, activation='sigmoid'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)
lr_anneal = ReduceLROnPlateau(monitor = 'val_acc', patience=5, factor=0.2, min_lr=1e-6)
mc = ModelCheckpoint('improved_cnn_model0.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)

model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test), callbacks=[es, mc, lr_anneal])

Epoch 1/20
1245/1250 [============================>.] - ETA: 0s - loss: 0.4244 - acc: 0.7980
Epoch 1: val_loss improved from inf to 0.32173, saving model to improved_cnn_model0.h5
1250/1250 [==============================] - 10s 8ms/step - loss: 0.4240 - acc: 0.7983 - val_loss: 0.3217 - val_acc: 0.8586 - lr: 0.0010
Epoch 2/20
1249/1250 [============================>.] - ETA: 0s - loss: 0.2765 - acc: 0.8875
Epoch 2: val_loss improved from 0.32173 to 0.29827, saving model to improved_cnn_model0.h5
1250/1250 [==============================] - 9s 7ms/step - loss: 0.2765 - acc: 0.8875 - val_loss: 0.2983 - val_acc: 0.8703 - lr: 0.0010
Epoch 3/20
1247/1250 [============================>.] - ETA: 0s - loss: 0.1936 - acc: 0.9263
Epoch 3: val_loss did not improve from 0.29827
1250/1250 [==============================] - 9s 7ms/step - loss: 0.1934 - acc: 0.9263 - val_loss: 0.3662 - val_acc: 0.8680 - lr: 0.0010
Epoch 4/20
1249/1250 [============================>.] - ETA: 0s - loss: 0.1283 - acc: 0

In [23]:
loaded_model = load_model('improved_cnn_model0.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

313/313 [==============================] - 1s 4ms/step - loss: 0.2983 - acc: 0.8703

 테스트 정확도: 0.8703


In [125]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dropout, Conv1D, GlobalMaxPooling1D, Dense, BatchNormalization, Reshape, Flatten
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam, Nadam
from tensorflow.keras.models import load_model

embedding_dim = 100 # 임베딩 벡터의 차원
dropout_ratio = 0.3 # 드롭아웃 비율
num_filters = 256 # 커널의 수
kernel_size = 3 # 커널의 크기
hidden_units = 128 # 뉴런의 수

model = Sequential()
model.add(Embedding(10000, embedding_dim))
model.add(Conv1D(num_filters, kernel_size, padding='valid', activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(1, activation='sigmoid'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)
lr_anneal = ReduceLROnPlateau(monitor = 'val_acc', patience=5, factor=0.2, min_lr=1e-6)
mc = ModelCheckpoint('improved_cnn_model12.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)

model.compile(optimizer=Nadam(learning_rate=0.001), loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test), callbacks=[es, mc, lr_anneal])

Epoch 1/20
1250/1250 [==============================] - ETA: 0s - loss: 0.3903 - acc: 0.8202
Epoch 1: val_loss improved from inf to 0.30564, saving model to improved_cnn_model12.h5
1250/1250 [==============================] - 6s 4ms/step - loss: 0.3903 - acc: 0.8202 - val_loss: 0.3056 - val_acc: 0.8694 - lr: 0.0010
Epoch 2/20
1244/1250 [============================>.] - ETA: 0s - loss: 0.2337 - acc: 0.9056
Epoch 2: val_loss did not improve from 0.30564
1250/1250 [==============================] - 5s 4ms/step - loss: 0.2335 - acc: 0.9057 - val_loss: 0.3058 - val_acc: 0.8703 - lr: 0.0010
Epoch 3/20
1246/1250 [============================>.] - ETA: 0s - loss: 0.1432 - acc: 0.9491
Epoch 3: val_loss did not improve from 0.30564
1250/1250 [==============================] - 5s 4ms/step - loss: 0.1432 - acc: 0.9490 - val_loss: 0.3317 - val_acc: 0.8715 - lr: 0.0010
Epoch 4/20
1243/1250 [============================>.] - ETA: 0s - loss: 0.0734 - acc: 0.9787
Epoch 4: val_loss did not improve from

In [122]:
loaded_model = load_model('improved_cnn_model1.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

313/313 [==============================] - 1s 3ms/step - loss: 0.3091 - acc: 0.8714

 테스트 정확도: 0.8714


In [127]:
loaded_model = load_model('improved_cnn_model12.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

313/313 [==============================] - 1s 2ms/step - loss: 0.3056 - acc: 0.8694

 테스트 정확도: 0.8694


In [93]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dropout, Conv1D, GlobalMaxPooling1D, Dense, BatchNormalization, Reshape
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model

# embedding_dim = 100 # 임베딩 벡터의 차원
# dropout_ratio = 0.3 # 드롭아웃 비율
# num_filters = 256 # 커널의 수
# kernel_size = 3 # 커널의 크기
# hidden_units = 128 # 뉴런의 수

# model = Sequential()
# model.add(Embedding(10000, embedding_dim))
# model.add(Dropout(dropout_ratio))
# model.add(Conv1D(num_filters, kernel_size, padding='valid', activation='relu'))
# model.add(BatchNormalization())
# model.add(Conv1D(num_filters, kernel_size, padding='valid', activation='relu'))
# #model.add(GlobalMaxPooling1D())
# model.add(Dense(hidden_units, activation='relu'))
# #model.add(Reshape((3,32)))
# model.add(Dropout(dropout_ratio))
# model.add(Conv1D(num_filters, kernel_size, padding='same', activation='relu'))
# model.add(BatchNormalization())
# model.add(Conv1D(num_filters, kernel_size, padding='same', activation='relu'))
# #model.add(GlobalMaxPooling1D())
# model.add(Dense(hidden_units, activation='relu'))
# #model.add(Reshape((3,32)))
# model.add(Dropout(dropout_ratio))
# model.add(Dense(1, activation='sigmoid'))


embedding_dim = 256 # 임베딩 벡터의 차원
dropout_ratio = 0.3 # 드롭아웃 비율
num_filters = 256 # 커널의 수
kernel_size = 3 # 커널의 크기
hidden_units = 128 # 뉴런의 수

model = Sequential()
model.add(Embedding(10000, embedding_dim))
model.add(Dropout(dropout_ratio))
model.add(Conv1D(num_filters, kernel_size, padding='valid', activation='relu'))
model.add(BatchNormalization())
model.add(Conv1D(num_filters, kernel_size, padding='valid', activation='relu'))
model.add(BatchNormalization())
model.add(Conv1D(num_filters, kernel_size, padding='valid', activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(hidden_units, activation='relu'))
model.add(Dropout(dropout_ratio))
model.add(Dense(1, activation='sigmoid'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)
lr_anneal = ReduceLROnPlateau(monitor = 'val_acc', patience=5, factor=0.2, min_lr=1e-6)
mc = ModelCheckpoint('improved_cnn_model11.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)

model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test), callbacks=[es, mc, lr_anneal])

Epoch 1/20
1246/1250 [============================>.] - ETA: 0s - loss: 0.4536 - acc: 0.7884
Epoch 1: val_loss improved from inf to 0.31900, saving model to improved_cnn_model11.h5
1250/1250 [==============================] - 12s 9ms/step - loss: 0.4534 - acc: 0.7885 - val_loss: 0.3190 - val_acc: 0.8621 - lr: 0.0010
Epoch 2/20
1250/1250 [==============================] - ETA: 0s - loss: 0.2995 - acc: 0.8774
Epoch 2: val_loss improved from 0.31900 to 0.31338, saving model to improved_cnn_model11.h5
1250/1250 [==============================] - 12s 9ms/step - loss: 0.2995 - acc: 0.8774 - val_loss: 0.3134 - val_acc: 0.8659 - lr: 0.0010
Epoch 3/20
1250/1250 [==============================] - ETA: 0s - loss: 0.2208 - acc: 0.9158
Epoch 3: val_loss did not improve from 0.31338
1250/1250 [==============================] - 12s 9ms/step - loss: 0.2208 - acc: 0.9158 - val_loss: 0.3210 - val_acc: 0.8743 - lr: 0.0010
Epoch 4/20
1250/1250 [==============================] - ETA: 0s - loss: 0.1605 - ac

In [ ]:
loaded_model = load_model('improved_cnn_model11.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

313/313 [==============================] - 1s 3ms/step - loss: 0.3134 - acc: 0.8659

 테스트 정확도: 0.8659


In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dropout, Conv1D, GlobalMaxPooling1D, Dense, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model

embedding_dim = 256 # 임베딩 벡터의 차원
dropout_ratio = 0.3 # 드롭아웃 비율
num_filters = 256 # 커널의 수
kernel_size = 3 # 커널의 크기
hidden_units = 128 # 뉴런의 수

model = Sequential()
model.add(Embedding(10000, embedding_dim))
model.add(Dropout(dropout_ratio))
model.add(Conv1D(num_filters, kernel_size, padding='valid', activation='relu'))
model.add(BatchNormalization())
model.add(Conv1D(num_filters, kernel_size, padding='valid', activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(hidden_units, activation='relu'))
model.add(Dropout(dropout_ratio))
model.add(Dense(1, activation='sigmoid'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)
#lr_anneal = ReduceLROnPlateau(monitor = 'val_acc', patience=5, factor=0.2, min_lr=1e-6)
mc = ModelCheckpoint('improved_cnn_model1.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test), callbacks=[es, mc])

Epoch 1/20
1250/1250 [==============================] - ETA: 0s - loss: 0.4279 - acc: 0.7985
Epoch 1: val_acc improved from -inf to 0.83980, saving model to improved_cnn_model1.h5
1250/1250 [==============================] - 10s 7ms/step - loss: 0.4279 - acc: 0.7985 - val_loss: 0.3508 - val_acc: 0.8398
Epoch 2/20
1249/1250 [============================>.] - ETA: 0s - loss: 0.2800 - acc: 0.8875
Epoch 2: val_acc improved from 0.83980 to 0.87140, saving model to improved_cnn_model1.h5
1250/1250 [==============================] - 9s 8ms/step - loss: 0.2802 - acc: 0.8875 - val_loss: 0.3091 - val_acc: 0.8714
Epoch 3/20
1249/1250 [============================>.] - ETA: 0s - loss: 0.2007 - acc: 0.9218
Epoch 3: val_acc did not improve from 0.87140
1250/1250 [==============================] - 9s 7ms/step - loss: 0.2007 - acc: 0.9218 - val_loss: 0.3389 - val_acc: 0.8622
Epoch 4/20
1248/1250 [============================>.] - ETA: 0s - loss: 0.1355 - acc: 0.9498
Epoch 4: val_acc did not improve fr

In [21]:
loaded_model = load_model('improved_cnn_model1.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

313/313 [==============================] - 1s 3ms/step - loss: 0.3091 - acc: 0.8714

 테스트 정확도: 0.8714


In [31]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dropout, Conv1D, GlobalMaxPooling1D, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model

embedding_dim = 256 # 임베딩 벡터의 차원
dropout_ratio = 0.3 # 드롭아웃 비율
num_filters = 256 # 커널의 수
kernel_size = 3 # 커널의 크기
hidden_units = 128 # 뉴런의 수

model = Sequential()
model.add(Embedding(10000, embedding_dim))
model.add(Dropout(dropout_ratio))
model.add(Conv1D(num_filters, kernel_size, padding='valid', activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(hidden_units, activation='relu'))
model.add(Dropout(dropout_ratio))
model.add(Dense(1, activation='sigmoid'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)
mc = ModelCheckpoint('improved_cnn_model3.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(optimizer='nadam', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test), callbacks=[es, mc])

Epoch 1/20
1241/1250 [============================>.] - ETA: 0s - loss: 0.3931 - acc: 0.8127
Epoch 1: val_acc improved from -inf to 0.86770, saving model to improved_cnn_model3.h5
1250/1250 [==============================] - 7s 5ms/step - loss: 0.3930 - acc: 0.8128 - val_loss: 0.3066 - val_acc: 0.8677
Epoch 2/20
1245/1250 [============================>.] - ETA: 0s - loss: 0.2352 - acc: 0.9056
Epoch 2: val_acc improved from 0.86770 to 0.87970, saving model to improved_cnn_model3.h5
1250/1250 [==============================] - 7s 5ms/step - loss: 0.2349 - acc: 0.9058 - val_loss: 0.2875 - val_acc: 0.8797
Epoch 3/20
1241/1250 [============================>.] - ETA: 0s - loss: 0.1416 - acc: 0.9479
Epoch 3: val_acc did not improve from 0.87970
1250/1250 [==============================] - 6s 5ms/step - loss: 0.1418 - acc: 0.9477 - val_loss: 0.3298 - val_acc: 0.8757
Epoch 4/20
1247/1250 [============================>.] - ETA: 0s - loss: 0.0753 - acc: 0.9738
Epoch 4: val_acc did not improve fro

In [32]:
loaded_model = load_model('improved_cnn_model3.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

313/313 [==============================] - 1s 2ms/step - loss: 0.2875 - acc: 0.8797

 테스트 정확도: 0.8797


In [39]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dropout, Conv1D, GlobalMaxPooling1D, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model

embedding_dim = 256 # 임베딩 벡터의 차원
dropout_ratio = 0.1 # 드롭아웃 비율
num_filters = 256 # 커널의 수
kernel_size = 3 # 커널의 크기
hidden_units = 128 # 뉴런의 수

model = Sequential()
model.add(Embedding(10000, embedding_dim))
model.add(Dropout(dropout_ratio))
model.add(Conv1D(num_filters, kernel_size, padding='valid', activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(hidden_units, activation='relu'))
model.add(Dropout(dropout_ratio))
model.add(Dense(1, activation='sigmoid'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
mc = ModelCheckpoint('improved_cnn_model4.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(optimizer='nadam', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test), callbacks=[es, mc])

Epoch 1/20
1246/1250 [============================>.] - ETA: 0s - loss: 0.3756 - acc: 0.8236
Epoch 1: val_acc improved from -inf to 0.86000, saving model to improved_cnn_model4.h5
1250/1250 [==============================] - 8s 5ms/step - loss: 0.3753 - acc: 0.8238 - val_loss: 0.3233 - val_acc: 0.8600
Epoch 2/20
1243/1250 [============================>.] - ETA: 0s - loss: 0.2076 - acc: 0.9182
Epoch 2: val_acc improved from 0.86000 to 0.86630, saving model to improved_cnn_model4.h5
1250/1250 [==============================] - 7s 5ms/step - loss: 0.2075 - acc: 0.9183 - val_loss: 0.3256 - val_acc: 0.8663
Epoch 3/20
1246/1250 [============================>.] - ETA: 0s - loss: 0.0948 - acc: 0.9675
Epoch 3: val_acc improved from 0.86630 to 0.87360, saving model to improved_cnn_model4.h5
1250/1250 [==============================] - 7s 5ms/step - loss: 0.0946 - acc: 0.9675 - val_loss: 0.3692 - val_acc: 0.8736
Epoch 4/20
1244/1250 [============================>.] - ETA: 0s - loss: 0.0381 - acc:

In [40]:
loaded_model = load_model('improved_cnn_model4.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

313/313 [==============================] - 1s 2ms/step - loss: 0.3692 - acc: 0.8736

 테스트 정확도: 0.8736


In [45]:
num_filters = 256 # 커널의 수
kernel_size = 3 # 커널의 크기

model = Sequential()
model.add(Dense(16, activation='relu', input_shape=(10000,)))
model.add(Dense(16, activation='relu'))
model.add(Conv1D(num_filters, kernel_size, padding='same', activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
mc = ModelCheckpoint('improved_cnn_model5.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.fit(X_train, y_train, epochs=20, batch_size=512, validation_data=(X_test, y_test), callbacks=[es, mc])
results = model.evaluate(X_test, y_test)

ValueError: Input 0 of layer "conv1d_38" is incompatible with the layer: expected min_ndim=3, found ndim=2. Full shape received: (None, 16)

In [55]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dropout, Conv1D, GlobalMaxPooling1D, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model

embedding_dim = 256 # 임베딩 벡터의 차원
dropout_ratio = 0.1 # 드롭아웃 비율
num_filters = 256 # 커널의 수
kernel_size = 3 # 커널의 크기
hidden_units = 128 # 뉴런의 수

model = Sequential()
model.add(Embedding(10000, embedding_dim))
model.add(Dropout(dropout_ratio))
model.add(Conv1D(num_filters, kernel_size, padding='same', activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Conv1D(num_filters, 2, padding='same', activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(hidden_units, activation='relu'))
model.add(Dropout(dropout_ratio))
model.add(Dense(1, activation='sigmoid'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
mc = ModelCheckpoint('improved_cnn_model5.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(optimizer='nadam', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test), callbacks=[es, mc])

ValueError: Input 0 of layer "conv1d_56" is incompatible with the layer: expected min_ndim=3, found ndim=2. Full shape received: (None, 256)

In [54]:
import tensorflow as tf

input_layer = tf.keras.layers.Input(shape=(100,), dtype=tf.int32)

layer = Embedding(10000, output_dim=embedding_dim)(input_layer)

layer_conv3 = Conv1D(num_filters, 3, activation="relu")(layer)
layer_conv3 = GlobalMaxPooling1D()(layer_conv3)

layer_conv4 = Conv1D(num_filters, 2, activation="relu")(layer)
layer_conv4 = GlobalMaxPooling1D()(layer_conv4)

layer = tf.keras.layers.concatenate([layer_conv4, layer_conv3], axis=1)
layer = BatchNormalization()(layer)
layer = Dropout(0.3)(layer)

output = Dense(16, activation="softmax")(layer)

model = tf.keras.Model(inputs=input_layer, outputs=output)

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
mc = ModelCheckpoint('improved_cnn_model5.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.fit(X_train, y_train, epochs=20, batch_size=512, validation_data=(X_test, y_test), callbacks=[es, mc])
results = model.evaluate(X_test, y_test)

Epoch 1/20


ValueError: in user code:

    File "/opt/conda/lib/python3.7/site-packages/keras/engine/training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "/opt/conda/lib/python3.7/site-packages/keras/engine/training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/opt/conda/lib/python3.7/site-packages/keras/engine/training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "/opt/conda/lib/python3.7/site-packages/keras/engine/training.py", line 860, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/opt/conda/lib/python3.7/site-packages/keras/engine/training.py", line 919, in compute_loss
        y, y_pred, sample_weight, regularization_losses=self.losses)
    File "/opt/conda/lib/python3.7/site-packages/keras/engine/compile_utils.py", line 201, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
  